
> **ISO2024 INTRODUCTORY SPATIAL 'OMICS ANALYSIS**
>
>
>- HYBRID : TORONTO & ZOOM
>- 9TH JULY 2024 <br>

>**Module 3 : Building your spatial model** ** PART I ** <BR>
>
>**Instructor : Shamini Ayyadhury**
>
---

> TOPICS COVERED

* A. Normalizing your data *
* B. Spatial and non-spatial clustering *

***

`
A. NORMALIZATION AND DIMENSIONAL REDUCTION
`

>>> IMPORT PACKAGES AND MANAGE PATHS TO YOUR FOLDERS AND FILES AS USUAL

In [ ]:
### import the following libraries. Some of these were used in the previous notebook and we are using them here as well
import sys # system specific parameters and functions
import os # operating system dependent functionality

import pandas as pd # data manipulation and analysis

import matplotlib.pyplot as plt # plotting library
import seaborn as sns # data visualization library based on matplotlib

import scanpy as sc # single-cell analysis in Python

sys.path.append('/home/shamini/data/projects/spatial_workshop/')
import pre_processing_fnc as ppf # import memory usage function only


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage


In [ ]:

### directory & filepaths

data_dir = '/home/shamini/data1/data_orig/data/spatial/xenium/10xGenomics/'
out = '/home/shamini/data/projects/spatial_workshop/out/'

os.makedirs(out+'module3/TgCRND8_17_8mths', exist_ok=True) # create a new directory to store the output files


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

>>> IMPORT ANNADATA FROM MODULE 2

1. Note that we will using the same anndata object that we had created for the TgCRND8 sample.
2. A supplementary script will process the wildtype sample and you will find this already saved in your module folder.

3. Prior to this module, cell_type labels from the allen dataset have been transferred over to the anndata objects of the samples you will use in this module.

4. You will now transfer the labels directly into the adata.obs slot.

In [ ]:

### IMPORT DATA

'''
1. ANNDATA OBJECT FROM MODULE 2
2. A PREVIOUSLY ANNOTATED CELL LABEL FOR THE MICE BRAIN - DERIVED FROM THE ALLEND BRAIN ATLAS
    *** students can look at supplementary script 01 to see how the cell label was processed
'''

### load previously processed AD mouse sample anndata object that we created from module 2
adata = sc.read_h5ad(out+'module2/TgCRND8_17_8mths/adata.h5ad')
print(adata)

### load the cell label for the AD mouse sample that was already created for you - see supplementary script 01
cell_label = pd.read_csv(out+'module3/allen_annotations/TgCRND8_allen.csv', index_col=0)
cell_label = cell_label['predicted.id']
print(cell_label)


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

In [ ]:
'''
ATTACH CELL LABELS TO THE ANNDATA OBJECT, BY CONCATENATING WITH THE OBSERVATION DATAFRAME
The number of cells labelled was missing slightly under 1000 cells, so we will drop the cells and their corresponding data from the anndata object
'''

# Subset adata to include only the cells present in cell_label
common_indices = adata.obs.index.intersection(cell_label.index)
adata = adata[common_indices].copy()

# Reindex cell_label to match adata_subset
cell_label_subset = cell_label.reindex(adata.obs.index)
adata.obs['cell_label'] = cell_label_subset

adata = adata[~adata.obs['cell_label'].isna()]


# Print results
print("\nSubset AnnData object created successfully!")
print(adata)


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

>>> STOP TO EXPLAIN THE CODE IN THE NEXT BLOCK

NORMALIZATION & DIMENSIONAL REDUCTION - BASIC STEPS 
1. We will will go through the primary steps of normalization - to keep the flow of the methods intact.
2. We will using 2 normalization methods 
    * Standard size factor normalization & lop1p transformation
    * Pearson residuals 
3. We will have a short discussion on the need for developmental in normalization and dimensional reduction methods before moving onto clustering 

In [ ]:
import numpy as np

adata.X = adata.raw.X.copy()

# 1. Pearson residual normalization
### perform the pearson residual transformation
adata.X = adata.raw.X.copy() ### reset the adata.X to the raw data
sc.experimental.pp.normalize_pearson_residuals(adata) ### perform the pearson residual transformation
adata.layers['pearson_residual'] = adata.X.copy() ### save the normalized and transformed adata.X in the layer

sc.pp.pca(adata, n_comps=9, svd_solver='arpack') ### perform PCA
adata.obsm['pearsonPCA'] = adata.obsm['X_pca'].copy() ### save the X_PCA coordinates to another layer labeled pearsonPCA

sc.pp.neighbors(adata, n_pcs=9, n_neighbors=15) ### find the neighbors
adata.uns['pearson_neigh'] = adata.uns['neighbors'] ### save the neighbors in another layer

sc.tl.umap(adata, neighbors_key='pearson_neigh', min_dist=0.1, spread=1.0, method='umap', init_pos='spectral') ### perform UMAP
adata.obsm['pearsonUMAP'] = adata.obsm['X_umap'].copy() ### save the UMAP coordinates to another layer labeled pearsonUMAP


# 2. Standard normalization
### now perform the standard normalization and transformation
adata.X = adata.raw.X.copy() ### reset the adata.X to the raw data
sc.pp.normalize_total(adata, target_sum=1e4) ### set the size factor to 1e4

sc.pp.log1p(adata) ### perform the log1p transformation
adata.layers['log1p_scaled'] = adata.X.copy() ### save the normalized and transformed adata.X in the layer

sc.pp.pca(adata, n_comps=9, svd_solver='arpack') ### perform PCA
adata.obsm['stdPCA'] = adata.obsm['X_pca'].copy() ### save the X_PCA coordinates to another layer labeled stdPCA

sc.pp.neighbors(adata, n_pcs=9, n_neighbors=15) ### find the neighbors
adata.uns['std_neigh'] = adata.uns['neighbors'] ### save the neighbors in another layer

sc.tl.umap(adata, neighbors_key='std_neigh', min_dist=0.1, spread=1.0, method='umap', init_pos='spectral') ### perform UMAP
adata.obsm['stdUMAP'] = adata.obsm['X_umap'].copy() ### save the UMAP coordinates to another layer labeled stdUMAP

#sc.tl.leiden(adata, resolution=0.3, key_added='std_leiden_0.3')


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes = axes.flatten()

fig.suptitle('A1. log1p vs Pearson residual normalization/tranformation', fontsize=18, y=1.05, x=0.3)

sc.pl.embedding(adata, basis='stdUMAP', 
                color='cell_label', 
                title='Standard Transformation', 
                legend_loc='on data', 
                ax=axes[0], 
                show=False)

sc.pl.embedding(adata, 
                basis='pearsonUMAP', 
                color='cell_label', 
                title='Pearson normalization', 
                legend_loc='on data', 
                ax=axes[1], 
                show=False)


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

---
>>> NOW YOU WILL EXPERIMENT WITH TWO ADDITIONAL METHODS OF NORMALIZATION/TRANSFORMATION

1. FIRST : YOU WILL REPEAT THE LOG1P TRANSFORMATION BUT BY CHANGING THE SIZE FACTOR USED TO NORMALIZE THE DATA.
2. SECOND : YOU WILL TRY A NEW METHOD CALLED THE FREEMAN-TUKEY TRANSFORMATION

3. THEN WE WILL REVIEW ALL 4 NORMALIZATION & TRANSFORMATION

In [ ]:

# 3. Median size factor normalization
### now perform the standard normalization and transformation

adata.X = adata.raw.X.copy() ### reset the adata.X to the raw data
median = np.median(adata.X.sum(axis=1)) ### calculate the median of the sum of the adata.X

scale_factors = median / adata.X.sum(axis=1) ### calculate the scale factors
adata.X = adata.X * (scale_factors[:, np.newaxis])  ### multiply the adata.X by the scale factors

sc.pp.log1p(adata) ### perform the log1p transformation
adata.layers['log1p_med'] = adata.X.copy() ### save the normalized and transformed adata.X in the layer

sc.pp.pca(adata, n_comps=9, svd_solver='arpack') ### perform PCA
adata.obsm['medPCA'] = adata.obsm['X_pca'].copy() ### save the X_PCA coordinates to another layer labeled stdPCA

sc.pp.neighbors(adata, n_pcs=9, n_neighbors=27) ### find the neighbors
adata.uns['med_neigh'] = adata.uns['neighbors'] ### save the neighbors in another layer

sc.tl.umap(adata, neighbors_key='med_neigh', min_dist=0.1, spread=1.0, method='umap', init_pos='spectral') ### perform UMAP
adata.obsm['medUMAP'] = adata.obsm['X_umap'].copy() ### save the UMAP coordinates to another layer labeled stdUMAP

#sc.tl.leiden(adata, resolution=0.3, key_added='std_leiden_0.3')


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

In [ ]:

# 4. Freeman Tukey transformation 
adata.X = adata.raw.X.copy() ### reset the adata.X to the raw data

### use numpy to perform the freeman tukey transformation
adata.X = np.sqrt(adata.X) + np.sqrt(adata.X+1) ### perform the freeman tukey transformation
adata.layers['freemanTukey'] = adata.X  ### save the normalized and transformed adata.X in the layer

sc.pp.pca(adata, n_comps=9, svd_solver='arpack') ### perform PCA
adata.obsm['freemanPCA'] = adata.obsm['X_pca'].copy() ### save the X_PCA coordinates to another layer labeled freemanPCA

sc.pp.neighbors(adata, n_pcs=9, n_neighbors=27) ### find the neighbors
adata.uns['freeman_neigh'] = adata.uns['neighbors'] ### save the neighbors in another layer
sc.tl.umap(adata, neighbors_key='freeman_neigh', min_dist=0.1, spread=1.0, method='umap', init_pos='spectral') ### perform UMAP
adata.obsm['freemanUMAP'] = adata.obsm['X_umap'].copy() ### save the UMAP coordinates to another layer labeled freemanUMAP


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

fig.suptitle('A2. Normalization and Transformation Methods', fontsize=18, y=1.05, x=0.3)
sc.pl.embedding(adata, basis='pearsonUMAP', 
                color='cell_label', 
                title='Pearson Residual Normalization', 
                legend_loc='on data', 
                ax=axes[0], 
                show=False)

sc.pl.embedding(adata, 
                basis='stdUMAP', 
                color='cell_label', 
                title='Standard Normalization', 
                legend_loc='on data', 
                ax=axes[1], 
                show=False)

sc.pl.embedding(adata, 
                basis='medUMAP', 
                color='cell_label', 
                title='Median Normalization', 
                legend_loc='on data', 
                ax=axes[2], 
                show=False)

sc.pl.embedding(adata, 
                basis='freemanUMAP', 
                color='cell_label', 
                title='Freeman Tukey Transformation', 
                legend_loc='on data', 
                ax=axes[3], 
                show=False)





#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

---
>>> STOP FOR DISCUSSION/LECTURE

*NORMALIZATION*
> 1. ARE CURRENT METHODS FOR NORMALIZING SINGLE CELL DATASETS RELEVENT FOR SPATIAL DATA NORMALIZATION?
> 2. WHAT ARE THE DIFFERENCES BETWEEN DROPLET/PLATE BASED SINGLE CELL SYSTEMS AND SINGLE MOLECULE IMAGING BASED SYSTEMS AND HOW DO YOU 
THINK THESE AFFECT THE COUNTS MODEL DISTRIBUTION AND SUBSEQUENT NORMALIATION METHODS?
> 3. WHAT ASPECTS OF SPATIAL MODELING COULD INFLUENCE THE DEVELOPMENT OF NOVEL NORMALIZATION METHODS FOR SPATIAL TECHNOLOGIES?

> *DIM REDUCTION METHODS*
> 1. WE ARE USING THE SCANPY PACKAGE DERIVED UMAP, PCA , NEAREST NEIGHBOURHOOD CALCULATION FOR UMAP AND LEIDEN. ARE THE PARAMTERS STILL VALID FOR AND OPTIMAL FOR SINGLE MOLECULE IMAGING PLATFORMS LIKE THE XENIUM, MERSCOPE ?
> 2. WHAT WOULD GIVE BETTER FLEXIBILITY IN EFFECTIVELY REDUCING DIMENSIONS AND IN IMPROVING EFFICIENCY?
>    * PARTICIPANTS ARE ENCOURAGED TO TRY OUT THESE ALTERNATIVE PACKAGES WHICH GIVES MORE FLEXIBILITY IN PARAMTER TUNING AND TO USE MACHINE LEARNING MODELING TO SPEED PROCESSING AS SINGLE MOLECULE SPATIAL DATASETS ARE MANIFOLD LARGER THAN SINGLE CELL DATASETS
---

>>> ADDITIONAL EXERCISE

NOW YOU WILL COMPARE THE FREEMAN TUKEY TRANSFORMATION USING COUNTS DATA HAS BEEN SCALED

In [ ]:
# 5. Freeman Tukey transformation AFTER scaling

### use numpy to perform the freeman tukey transformation

adata.X = adata.raw.X.copy() ### reset the adata.X to the raw data
median = np.median(adata.X.sum(axis=1)) ### calculate the median of the sum of the adata.X

scale_factors = median / adata.X.sum(axis=1) ### calculate the scale factors
adata.X = adata.X * (scale_factors[:, np.newaxis])  ### multiply the adata.X by the scale factors

adata.X = np.sqrt(adata.X) + np.sqrt(adata.X+1) ### perform the freeman tukey transformation
adata.layers['freemanTukey_medN'] = adata.X  ### save the normalized and transformed adata.X in the layer

sc.pp.pca(adata, n_comps=9, svd_solver='arpack') ### perform PCA
adata.obsm['freeman_medN_PCA'] = adata.obsm['X_pca'].copy() ### save the X_PCA coordinates to another layer labeled freemanPCA

sc.pp.neighbors(adata, n_pcs=9, n_neighbors=27) ### find the neighbors
adata.uns['freeman_medN_neigh'] = adata.uns['neighbors'] ### save the neighbors in another layer
sc.tl.umap(adata, neighbors_key='freeman_medN_neigh', min_dist=0.1, spread=1.0, method='umap', init_pos='spectral') ### perform UMAP
adata.obsm['freeman_medN_UMAP'] = adata.obsm['X_umap'].copy() ### save the UMAP coordinates to another layer labeled freemanUMAP


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

fig.suptitle('A3. Freeman Tukey Transformation with and without scaling', fontsize=18, y=1.05, x=0.3)

sc.pl.embedding(adata, 
                basis='freemanUMAP', 
                color='cell_label', 
                title='Freeman Tukey Transformation after median', 
                legend_loc='on data', 
                ax=axes[0], 
                show=False)

sc.pl.embedding(adata, 
                basis='freeman_medN_UMAP', 
                color='cell_label', 
                title='med size factor normalization followed by Freeman Tukey Transformation', 
                legend_loc='on data', 
                ax=axes[1], 
                show=False)



#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

---
>>> STOP FOR DISCUSSION/LECTURE

1. The point of the above exercise is NOT to show that normalization and transformation is important for visualization.
2. The visualization tools we have used here - UMAP - is simply a medium to understand the concept.
3. The main point here is - the choice of a normalization methods depends on many factors 
    * dataset & cell-types
    * the plexity of your dataset
    * platform - imaging versus sequencing 
4. Recall now your lessons from module 2
    * Are the normalization methods we have used here appropriate?
    * What additional considerations do we need to consider when normalizing image-based datasets?

Remember - you are building a model <br>
    * there are good models and worse models - but no wrong or right models <br>
    * choose the model that gives you the best vantage point to answer your questions.



In [ ]:
#adata.write_h5ad(out+'module3/TgCRND8_17_8mths/adata_module3a.h5ad') ### save the anndata object

---
>>> REVIEW FOR PART II

> END OF MODULE 3 : NORMALIZATION, DIMENSIONAL REDUCTION - PART 1 <br>
> Thank you and see you in the next lecture where we will tackle spatial clustering